In [23]:
import numpy as np

#creating a positive definite matrix is not so easy, I think!
toy_Sigma=np.array([[-5,2,0],[2,1,-1],[0,-1,3]],dtype=float)
toy_Sigma=toy_Sigma.T.dot(toy_Sigma)
print(np.linalg.det(toy_Sigma))
toy_mu=np.array([-2,2,3])
toy_missing_indices=np.array([0,2])
toy_X=np.random.randint(-5,5,30).reshape(10,3).astype(np.float32)
print(toy_Sigma,"\n",toy_X)

483.9999999999998
[[29. -8. -2.]
 [-8.  6. -4.]
 [-2. -4. 10.]] 
 [[ 3.  1.  0.]
 [-1. -2.  0.]
 [-1. -1.  1.]
 [ 1.  3.  3.]
 [-1.  0. -5.]
 [-4.  0.  1.]
 [ 1.  2.  4.]
 [-1.  0.  2.]
 [ 3.  2. -3.]
 [-2.  3.  2.]]


In [2]:
A=np.array([[1,2,5],[4,5,6],[7,8,9]])
B=np.array([[1,-2,3],[-4,5,-6],[-7,8,11]])
print(B)
print(A.dot(np.linalg.solve(A,B)))
print(np.linalg.det(A))

[[ 1 -2  3]
 [-4  5 -6]
 [-7  8 11]]
[[ 1. -2.  3.]
 [-4.  5. -6.]
 [-7.  8. 11.]]
-6.0


In [24]:
def calculate_C(Sigma,missing_indices,verbose=False):
    p=Sigma.shape[0]
    m=missing_indices.shape[0]
    if verbose:
        print("Original unconditional covariance : \n",Sigma)
    observed_indices=np.arange(p)[~np.isin(np.arange(p),missing_indices)]
    appended_indices=np.append(observed_indices, missing_indices)
    Sigma=(Sigma[appended_indices,:])[:,appended_indices]
    
    if verbose:
        print("Unconditional covariance rearranged so that missing indices are at the bottom : \n",Sigma)
    Sigma11=Sigma[0:(p-m),0:(p-m)]
    Sigma21=Sigma[(p-m):p,0:(p-m)]
    Sigma22=Sigma[(p-m):p,(p-m):p]
    Sigma22_conditional=Sigma22-Sigma21.dot(np.linalg.solve(Sigma11,Sigma21.T))
    
    reverse_permutation=np.argsort(appended_indices)
    result=np.full((p,p),0)
    result[(p-m):p,(p-m):p]=Sigma22_conditional
    if verbose:
        print("The conditional covariance when ordered is : \n",result)
    result=(result[reverse_permutation,:])[:,reverse_permutation]
    if verbose:
        print("After permuting back we get : \n", result)
    return(result)
    #print(Sigma,appended_indices,observed_indices,missing_indices,Sigma22,Sigma12,Sigma11)

In [25]:
#print(~np.isin(np.arange(3),toy_missing_indices))
calculate_C(toy_Sigma,toy_missing_indices,verbose=True)


Original unconditional covariance : 
 [[29. -8. -2.]
 [-8.  6. -4.]
 [-2. -4. 10.]]
Unconditional covariance rearranged so that missing indices are at the bottom : 
 [[ 6. -8. -4.]
 [-8. 29. -2.]
 [-4. -2. 10.]]
The conditional covariance when ordered is : 
 [[ 0  0  0]
 [ 0 18 -7]
 [ 0 -7  7]]
After permuting back we get : 
 [[18  0 -7]
 [ 0  0  0]
 [-7  0  7]]


array([[18,  0, -7],
       [ 0,  0,  0],
       [-7,  0,  7]])

In [29]:
def lcomp(mu,Sigma, X):
    N=X.shape[0]
    p=X.shape[1]
    result=-N/2*np.log(np.linalg.det(Sigma))
    X_muT=X-mu.T
    for n in range(N):
        result=result+(X[n,:]-mu.T).dot(np.linalg.solve(Sigma, X[n,:].T-mu)) #is repetitively summing here bad?
    return result

In [30]:
lcomp(toy_mu,toy_Sigma,toy_X)

39.13503001187138

In [35]:
def calculate_xhat(xn,mu,Sigma,missing_indices):
    #m=missing_indices.shape[0]
    p=Sigma.shape[0]
    
    xhatn=xn.copy()
    observed_indices=np.arange(p)[~np.isin(np.arange(p),missing_indices)]
    #appended_indices=np.append(observed_indices, missing_indices)
    
    Sigma22=Sigma[missing_indices,:][:,missing_indices]
    Sigma11=Sigma[observed_indices,:][:,observed_indices]
    Sigma21=Sigma[missing_indices,:][:,observed_indices]
    
    mu1=mu[observed_indices]
    mu2=mu[missing_indices]
    
    mu2_conditional=mu2-Sigma21.dot(np.linalg.solve(Sigma11,mu1))
    #print(xhatn)
    #print(mu2_conditional,xhatn[missing_indices,],mu2_conditional.shape,xhatn[missing_indices,].shape)
    xhatn[missing_indices,] = mu2_conditional
    #this doesnt seem to work 
    #reason: Array is passed as a view, potential danger!
    #hmm still doesnt work wtf
    #Its because a numpy array of ints rounds floats that are added to it wtfffff, fixed now
    
    #print(xhatn.base)
    #print(mu2_conditional,xhatn[missing_indices])
    
    return xhatn

In [36]:
print(calculate_xhat(toy_X[0,:],toy_mu,toy_Sigma,toy_missing_indices))
print(toy_X[0,:])

[0.6666667 1.        4.3333335]
[3. 1. 0.]


In [19]:
test=np.array([1,2,3])
thing=test
thing[np.array([0,2])]=np.array([-1.2,5.7])
print(thing)

[-1  2  5]
